# 基于预期差的反转策略

一个典型的**量化投资Alpha因子构造公式**，用于生成股票选择信号，核心逻辑是“**基于预期差的反转策略**”。


## 一、表达式构造逻辑

### 1.1 表达式结构（从内到外）

```bash
zscore(
  rank(
    ts_delta(
      winsorize(
        ts_backfill(
          vec_sum(anl4_mark), 
          120
        ), 
        std=4
      ), 
      30
    ) * (-1 * ts_rank(close, 10))
  )
)
```


### 1.2 逐层函数解释
#### 1.2.1. 最内层：`vec_sum(anl4_mark)`
- **函数**：向量求和（`vec_sum`）
- **参数**：`anl4_mark`（推测是某种分析指标的向量，如分析师情绪、技术指标组合等）
- **作用**：将多维度的`anl4_mark`压缩为单一数值，简化后续计算。


#### 1.2.2. 第二层：`ts_backfill(vec_sum(anl4_mark), 120)`
- **函数**：时间序列回填（`ts_backfill`）
- **参数**：
  - 第一个参数：`vec_sum(anl4_mark)`的结果（标量）
  - 第二个参数：`120`（回填窗口，单位通常为交易日）
- **作用**：将当前时刻的`anl4_mark`求和值，回填到过去120个交易日，生成一个长度为120的时间序列。例如，若当前是`t`时刻，则`t-1`、`t-2`、…、`t-120`时刻的值都被填充为`t`时刻的求和结果。


#### 1.2.3. 第三层：`winsorize(ts_backfill(...), std=4)`
- **函数**：缩尾处理（`winsorize`）
- **参数**：
  - 第一个参数：`ts_backfill`的结果（长度为120的时间序列）
  - 第二个参数：`std=4`（标准差倍数，用于确定缩尾阈值）
- **作用**：对回填后的时间序列进行“缩尾”，将超过**均值±4倍标准差**的极端值替换为阈值，降低异常值对后续计算的影响。


#### 1.2.4. 第四层：`ts_delta(winsorize(...), 30)`
- **函数**：时间序列差分（`ts_delta`）
- **参数**：
  - 第一个参数：`winsorize`的结果（经过缩尾的时间序列）
  - 第二个参数：`30`（差分窗口，单位为交易日）
- **作用**：计算时间序列的**30天变化量**（即当前值 - 30天前的值），用于衡量`anl4_mark`求和值的“短期趋势变化”。


#### 1.2.5. 第五层：`(-1 * ts_rank(close, 10))`
- **函数**：时间序列排名（`ts_rank`）
- **参数**：
  - 第一个参数：`close`（收盘价）
  - 第二个参数：`10`（排名窗口，单位为交易日）
- **作用**：
  - 计算当前收盘价在过去10个交易日中的排名（值越高，说明当前价格在近期越高）；
  - 乘以`-1`后，得到“**短期反转信号**”（值越高，说明近期股价下跌越多）。


#### 1.2.6. 第六层：`ts_delta(...) * (-1 * ts_rank(close, 10))`
- **作用**：将`anl4_mark`的30天变化量（代表“预期变化”）与股价的10天反转信号（代表“市场表现”）相乘，构建**预期差因子**：
  - 若`anl4_mark`变化量为正（预期改善），且股价近期下跌（反转信号为正），则乘积为正（买入信号）；
  - 若`anl4_mark`变化量为负（预期恶化），且股价近期上涨（反转信号为负），则乘积为负（卖出信号）。


#### 1.2.7. 第七层：`rank(...)`
- **函数**：截面排名（`rank`）
- **作用**：对所有股票的“预期差因子”进行**截面排名**（即当天所有股票按因子值从低到高排序，赋予1到N的排名），将因子值转换为相对排名，消除量纲影响。


#### 1.2.8. 最外层：`zscore(...)`
- **函数**：标准化（`zscore`）
- **作用**：对截面排名后的因子值进行**Z-score标准化**（即减去均值，除以标准差），将排名转换为均值为0、标准差为1的正态分布，便于后续组合优化和风险控制。


### 1.3 核心逻辑总结
这个表达式的本质是：**寻找“预期改善但近期股价下跌”的股票，或“预期恶化但近期股价上涨”的股票**，通过预期与市场表现的背离来获取超额收益。


### 1.4 适用场景
- 适合**中短期量化策略**（信号周期为10-30天）；
- 常用于**股票多空组合**（多头配置因子值高的股票，空头配置因子值低的股票）；
- 适合**高流动性股票池**（如沪深300、中证500），因为反转信号在流动性好的股票中更有效。


### 1.5 潜在问题与优化方向
1. **`ts_backfill(120)`的合理性**：将当前值回填120天可能导致信号“固化”，建议缩短回填窗口（如30天）或改用衰减回填（`decay`参数）；
2. **因子相关性**：需验证该因子与其他常用因子（如市值、估值）的相关性，避免因子冗余；
3. **换手率控制**：短期反转信号可能导致高换手率，可通过增加持有期或平滑因子值来降低交易成本。


### 1.6 WorldQuant平台适配性
- 该表达式符合WorldQuant平台的函数规范，可直接用于回测；
- 建议添加**中性化处理**（如市值中性、行业中性），避免因子暴露于系统性风险；
- 回测时关注`Fitness`、`Sharpe`和`PC`（Prod Correlation）指标，确保因子有效性和低相关性。


总结：这个表达式是一个基于“预期差+反转”逻辑的Alpha因子，通过多层时间序列处理和截面标准化，生成可用于股票选择的量化信号。